In [2]:
# https://stages.ai/competitions/74/discussion/talk/post/475

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
import os, cv2

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mtcnn = MTCNN(keep_all=True, device=device)
new_img_dir = '/opt/ml/input/data/train/face_crop'
img_path = '/opt/ml/input/data/train/images'

cnt = 0

for paths in os.listdir(img_path):
    if paths[0] == '.': continue
    
    sub_dir = os.path.join(img_path, paths) # sub dir = 사람
    
    for imgs in os.listdir(sub_dir):
        if imgs[0] == '.': continue
        
        img_dir = os.path.join(sub_dir, imgs)
        img = cv2.imread(img_dir)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        #mtcnn 적용
        boxes,probs = mtcnn.detect(img)
        
        # boxes 확인
        if len(probs) > 1: 
            print(boxes)
        if not isinstance(boxes, np.ndarray):
            print('Nope!')
            # 직접 crop
            img=img[100:400, 50:350, :]
        
        # boexes size 확인
        else:
            xmin = int(boxes[0, 0])-30
            ymin = int(boxes[0, 1])-30
            xmax = int(boxes[0, 2])+30
            ymax = int(boxes[0, 3])+30
            
            if xmin < 0: xmin = 0
            if ymin < 0: ymin = 0
            if xmax > 384: xmax = 384
            if ymax > 512: ymax = 512
            
            img = img[ymin:ymax, xmin:xmax, :]
            
        tmp = os.path.join(new_img_dir, paths)
        cnt += 1
        os.makedirs(tmp, exist_ok = True)
        plt.imsave(imgs, img)
        
print(cnt)

[[ 98.7136  141.22815 283.81116 358.53503]
 [112.23799 265.19415 249.70847 425.5892 ]]
[[125.939926 136.23955  274.69144  328.32425 ]
 [164.3257   322.4367   239.00107  400.84625 ]]
[[ 94.49077 169.36238 246.71709 337.88666]
 [105.26928 303.6895  241.28395 442.20517]]
Nope!
Nope!
Nope!
Nope!
[[ 81.13768  130.26694  306.03076  419.31436 ]
 [ 73.827774 265.17657  294.4686   510.65256 ]]
Nope!
Nope!
[[ 82.19105 131.1393  300.32233 391.3126 ]
 [123.91324 376.49283 260.99106 555.2465 ]]
[[105.4529   167.2945   296.11792  395.76816 ]
 [ 98.029495 310.92334  277.90698  543.27496 ]]
Nope!
Nope!
Nope!
[[113.5807  144.2197  223.23032 264.9243 ]
 [167.57397 166.69429 267.60992 278.3009 ]]
Nope!
[[113.333984 144.49509  272.01868  359.02905 ]
 [143.39404  342.15176  245.25626  448.36676 ]]
[[128.44455 190.01183 235.35612 329.7388 ]
 [148.31401 287.2991  257.72565 418.3135 ]]
[[127.17657 194.34232 239.0916  328.70508]
 [154.35901 288.39603 250.12537 407.39725]]
Nope!
[[138.38205 163.79243 312.26962 

In [10]:
import pandas as pd

ff = pd.read_csv('/opt/ml/input/data/train/train_revised.csv')
ff.head(20)

,id,gender,race,age,path,class
0,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,16
1,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,4
2,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,4
3,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,10
4,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,4
5,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,4
6,000002,female,Asian,52,/opt/ml/input/data/train/images/000002_female_...,4
7,000005,female,Asian,58,/opt/ml/input/data/train/images/000005_female_...,16
8,000005,female,Asian,58,/opt/ml/input/data/train/images/000005_female_...,4
9,000005,female,Asian,58,/opt/ml/input/data/train/images/000005_female_...,4


In [11]:
import re
new= pd.read_csv('/opt/ml/input/data/train/train_total.csv')
new['path'] = new['path'].map(lambda x : re.sub(r'images', 'face_crop',x))
new.head()

,path,class
0,/opt/ml/input/data/train/face_crop/000002_fema...,16
1,/opt/ml/input/data/train/face_crop/000002_fema...,4
2,/opt/ml/input/data/train/face_crop/000002_fema...,4
3,/opt/ml/input/data/train/face_crop/000002_fema...,10
4,/opt/ml/input/data/train/face_crop/000002_fema...,4


In [12]:
new.to_csv('/opt/ml/input/data/train/train_total_facecrop.csv', index = False)

In [ ]:
# kaggle 
import os, cv2
from facenet_pytorch import MTCNN
from PIL import Image
import pandas as pd

if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print('running in cuda')
else:
    device = torch.device("cpu")
    print('running in cpu')
    
df = pd.read_csv('')
mtcnn = MTCNN(keep_all=True, device=device)
idx_list = []
remove_img_list = []
remove_img_name = []

root = ''
new_path = ''
for i in range(len(df['GENDER'])):
    flag = False
    if not 10 < int(df['AGE'].iloc[i]) < 100:
        idx_list.append(i)
        flag = True
    elif df['GENDER'].iloc[i] != 'MALE' and df['GENDER'].iloc[i] != 'FEMALE':
        idx_list.append(i)
        flag = True
    img_path = os.path.join(root, df['name'].iloc[i])
    if flag:
        os.remove(os.path.join(root, df['name'].iloc[i]))
    else:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        boxes,probs = mtcnn.detect(img)
        
        if not isinstance(boxes, np.ndarray):
            os.remove(img_path)
        else:
            xmin = int(boxes[0, 0])-30
            ymin = int(boxes[0, 1])-30
            xmax = int(boxes[0, 2])+30
            ymax = int(boxes[0, 3])+30
            
            if xmin < 0: xmin = 0
            if ymin < 0: ymin = 0
            if xmax > 384: xmax = 384
            if ymax > 512: ymax = 512
            
            img = img[ymin:ymax, xmin:xmax, :]
            img = cv2.resize(img, (224,224))
            plt.imsave(os.path.join(new_path, df['name'].iloc[i]), img)
            os.remove(os.path.join(img_path))
    
    
df = df.drop(idx_list)
df.to_csv('/opt/ml/new_csv.csv', index=False)